## Submission Deadline- 8th October'17, 11:00 PM.

### Assignment Topic: Making Election Predictions  

If during the early part of 2014 you had to predict the upcoming Lok Sabha Elections, what would be your predictive model?  (Say you wanted to predict the election outcomes six months before the elections were held)

#### To solve this problem…here are some possible stuff you can think of….

* Identify data sources. This will depend upon what factors you think can determine winning elections
* Identifying the right technique/techniques to model that data
* Make predictions, validate and update your model.

#### Submission details:

##### Part 1: Identify the data sources, the variables of interest. Process the data and create a ‘clean’ data file(s).

Marks distribution: 20.

** What needs to be submitted: **
* Text file explaining what variables you are choosing and why, the source codes, explanation etc
* The cleaned data file (s)
* Relevant r codes if any
Marks will be awarded based on what factors you think are important in model building, what data sources you identify and finally how the clean data looks. The clean data has to be submitted as CSV (files).

##### Part 2: Develop the model,present a predictive model.
Marks distribution: 20

** What needs to be submitted: **
* Text file explaining the model (s), why did you do what you did, model performance et
* Relevant R codes if any



## Data Sources are 
### http://eci.nic.in/eci_main1/ElectionStatistics.aspx
The above data source will help us in getting candidates electoral inforormation. As the Question asks about "redict the upcoming Lok Sabha Elections" and does not specify id it is at the particular  candidate level or party level. We will assume it as a candidate level and also assume that Candidates are anaounced for by all parties in all constituency.

** We will take the winning information of 2009 and use as one of the predeicting parameter for 2014. **

#### 2009 data : http://eci.nic.in/eci_main/StatisticalReports/candidatewise/GE2009.xls [ Sheets : electors, Cand_Wise ]
#### 2014 data : http://eci.nic.in/eci_main/StatisticalReports/candidatewise/LS-2014_ElectionResult.xls [Sheets : electors, Cand_Wise ]

#### https://raw.githubusercontent.com/datameet/india-election-data/master/affidavits/myneta.2014.csv
Criminal Cases,Education,ID,Party,Sno,Total Assets,Total Liabilities,Year,ST_CODE,PC_CODE

#### Source Data

In [1]:
library(stringr)
library(MASS)
library(stringdist)

In [2]:
getwd()
setwd("C:/Home/Work/GreatLakes/Machine Learning/Assignment")
#setwd(dir = "D:/")

[1] "C:/Home/Work/GreatLakes/Machine Learning/Assignment"

In [3]:
LS2009Electors = read.csv(file = "LS2009Electors.csv", header = T, sep = ",")
LS2009Candidates = read.csv(file = "LS2009Candidate.csv", header = T, sep = ",")
LS2014Electors = read.csv(file = "LS2014Electors.csv", header = T, sep = ",")
LS2014Candidates = read.csv(file = "LS2014Candidate.csv", header = T, sep = ",")
Candidate2014Declaration = read.csv(file = "MyNeta.csv", header = T, sep = ",")

In [4]:
names(LS2009Electors)[1]='STATE.CODE'
names(LS2014Electors)[1] = 'STATE.CODE'

In [5]:
unique(LS2009Electors$STATE)

[1] Andhra Pradesh            Arunachal Pradesh        
 [3] Assam                     Bihar                    
 [5] Goa                       Gujarat                  
 [7] Haryana                   Himachal Pradesh         
 [9] Jammu & Kashmir           Karnataka                
[11] Kerala                    Madhya Pradesh           
[13] Maharashtra               Manipur                  
[15] Meghalaya                 Mizoram                  
[17] Nagaland                  Orissa                   
[19] Punjab                    Rajasthan                
[21] Sikkim                    Tamil Nadu               
[23] Tripura                   Uttar Pradesh            
[25] West Bengal               Chattisgarh              
[27] Jharkhand                 Uttarakhand              
[29] Andaman & Nicobar Islands Chandigarh               
[31] Dadra & Nagar Haveli      Daman & Diu              
[33] NCT OF Delhi              Lakshadweep              
[35] Puducherry               
35 Levels: Andaman & Nicobar Islands Andhra Pradesh Arunachal Pradesh ... West Bengal

#### We will analyze the results for South India, by considering following states (Kerala, Tamil Nadu, Pondicherry, Karnataka and Andra Pradesh), We will assume that post-election 2014 also united andra existed. We will make sure the agrigate count of candidates matches with UP or more than that.

In [6]:
Candidate.Freequency = table(LS2014Candidates$State.name)
Candidate.Freequency = as.data.frame(Candidate.Freequency)
Candidate.Freequency = Candidate.Freequency[with(Candidate.Freequency, order(-Freq)),]

##### UP

In [7]:
UP = Candidate.Freequency[Candidate.Freequency$Var1 == 'Uttar Pradesh',]
UP_candidates = UP$Freq
UP_candidates

[1] 1368

##### Southindia Candidates

In [8]:
SI = Candidate.Freequency[Candidate.Freequency$Var1 %in% c('Kerala','Karnataka','Andhra Pradesh','Tamil Nadu','Puducherry'),]
si_ctr=0
for(si in 1:nrow(SI))
{
  si_ctr = si_ctr + SI$Freq[si]
}
SI_Candidates = si_ctr

Cadidates = data.frame(UP_candidates,SI_Candidates)
names(Cadidates) = c('Uttar Pradesh','South India')
Cadidates

Uttar Pradesh,South India
1368,2306


###  We will choose South Indian states for analysis

#### Before proceeding any further, it is always better to keep the back-up of originial data.

In [9]:
SRC_LS2009Electors = LS2009Electors
SRC_LS2009Candidates = LS2009Candidates
SRC_LS2014Electors = LS2014Electors
SRC_LS2014Candidates = LS2014Candidates
SRC_Candidate2014Declaration = Candidate2014Declaration

### Filtering the Data for only Southindian States

In [10]:
LS2009Electors = LS2009Electors[LS2009Electors$STATE.CODE %in% c('S01','S10','S11','S22','U07'),]
LS2009Candidates = LS2009Candidates[LS2009Candidates$ST_CODE %in% c('S01','S10','S11','S22','U07'),]
LS2014Electors = LS2014Electors[LS2014Electors$STATE.CODE %in% c('S01','S10','S11','S22','U07'),]
LS2014Candidates = LS2014Candidates[LS2014Candidates$ST_CODE %in% c('S01','S10','S11','S22','U07'),]
Candidate2014Declaration = Candidate2014Declaration[Candidate2014Declaration$ST_CODE %in% c('S01','S10','S11','S22','U07'),]

##### Dimention Check

In [11]:
dim(LS2009Electors)

[1] 130   8

In [12]:
dim(LS2014Electors)

[1] 130   7

In [13]:
dim(LS2009Candidates)

[1] 2064   14

In [14]:
dim(LS2014Candidates)

[1] 2306   14

In [15]:
dim(Candidate2014Declaration)

[1] 2156   12

###### Observations : in 2009 thre are 8 variables for electoral information, but we have only 7 variables in 2014.

## Data Cleaning and Data Preparation

#### first level Data cleaning - Converting the STATE and PARLIAMENTARY.CONSTITUENCY and also trimming white spaces in electoral data

### LS2009Electors

In [16]:
str(LS2009Electors)

'data.frame':	130 obs. of  8 variables:
 $ STATE.CODE                : Factor w/ 35 levels "S01","S02","S03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ STATE                     : Factor w/ 35 levels "Andaman & Nicobar Islands",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ PC.NO                     : int  1 2 3 4 5 6 7 8 9 10 ...
 $ PARLIAMENTARY.CONSTITUENCY: Factor w/ 540 levels "Adilabad","Agra",..: 1 405 272 385 540 343 331 459 218 121 ...
 $ Total.voters              : int  864165 905332 990646 891508 1017372 1061993 1206223 865357 732212 1086510 ...
 $ Total_Electors            : int  1131211 1315642 1496211 1333271 1359566 1389721 2343050 1574818 1393242 1681664 ...
 $ TOT_CONTESTANT            : int  9 15 15 12 10 9 19 23 23 15 ...
 $ POLL.PERCENTAGE           : num  76.4 68.8 66.2 66.9 74.8 ...


In [17]:
LS2009Electors$STATE = trimws(tolower(LS2009Electors$STATE))
LS2009Electors$PARLIAMENTARY.CONSTITUENCY = trimws(tolower(LS2009Electors$PARLIAMENTARY.CONSTITUENCY))
cleaned.PARLIAMENTARY.CONSTITUENCY = sapply(LS2009Electors$PARLIAMENTARY.CONSTITUENCY, function(x) gsub('[ ]{2,}',' ',x))
LS2009Electors$PARLIAMENTARY.CONSTITUENCY = cleaned.PARLIAMENTARY.CONSTITUENCY

##### Sort the Data

In [18]:
LS2009Electors = LS2009Electors[with(LS2009Electors, order(STATE.CODE,PC.NO)),]

### LS2014Electors

In [19]:
str(LS2014Electors)

'data.frame':	130 obs. of  7 variables:
 $ STATE.CODE                : Factor w/ 35 levels "S01","S02","S03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ STATE                     : Factor w/ 35 levels "Andaman & Nicobar Islands",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ PC.NO                     : int  1 2 3 4 5 6 7 8 9 10 ...
 $ PARLIAMENTARY.CONSTITUENCY: Factor w/ 540 levels "Adilabad ","Agra",..: 1 405 272 385 540 343 331 459 218 121 ...
 $ Total.voters              : int  1055593 1025194 1127225 1034032 1099784 1193548 1624859 1004763 971770 1322312 ...
 $ Total_Electors            : int  1386282 1425355 1550810 1496193 1445354 1536166 3183083 1893741 1823217 2185164 ...
 $ POLL.PERCENTAGE           : num  76.2 71.9 72.7 69.1 76.1 ...


In [20]:
LS2014Electors$STATE = trimws(tolower(LS2014Electors$STATE))
LS2014Electors$PARLIAMENTARY.CONSTITUENCY = trimws(tolower(LS2014Electors$PARLIAMENTARY.CONSTITUENCY))
cleaned.PARLIAMENTARY.CONSTITUENCY = sapply(LS2014Electors$PARLIAMENTARY.CONSTITUENCY, function(x) gsub('[ ]{2,}',' ',x))
LS2014Electors$PARLIAMENTARY.CONSTITUENCY = cleaned.PARLIAMENTARY.CONSTITUENCY


##### Sort the Data

In [21]:
LS2014Electors = LS2014Electors[with(LS2014Electors, order(LS2014Electors$STATE.CODE,LS2014Electors$PC.NO)),]

### Our Next step will be Joining the 2014 Electors with 2009 and capture previous time's Election result and study if it has an influence

In [22]:
temp2009 = LS2009Electors
temp2014 = LS2014Electors
names(temp2009) = c("STATE.CODE_2009","STATE_2009","PC.NO_2009","PARLIAMENTARY.CONSTITUENCY_2009","Total.voters_2009","Total_Electors_2009","TOT_CONTESTANT_2009","POLL.PERCENTAGE_2009")
names(temp2014) = c("STATE.CODE_2014","STATE_2014","PC.NO_2014","PARLIAMENTARY.CONSTITUENCY_2014","Total.voters_2014","Total_Electors_2014","POLL.PERCENTAGE_2014")
temp = merge(x=temp2009,y=temp2014,by.x=c("STATE.CODE_2009","PC.NO_2009","PARLIAMENTARY.CONSTITUENCY_2009"), by.y = c("STATE.CODE_2014","PC.NO_2014","PARLIAMENTARY.CONSTITUENCY_2014"))

** We will find list of Constituency which has inconsistent naming convention between 2009 and 2014. and we match 2014 to match 2009.. This only for Electoral data for now and will do the same for Candidates. **

In [23]:
t_stc = c()
t_pcn = c()
t_pc09 = c()
t_pc14 = c()
t_rn = c()
for(ii in 1:nrow(temp2014))
{
  if(!(temp2014$PARLIAMENTARY.CONSTITUENCY_2014[ii] %in% temp$PARLIAMENTARY.CONSTITUENCY_2009))
  {
    t_rn = append(t_rn, ii)
    t_stc = append(t_stc,temp2014$STATE.CODE_2014[ii])
    t_pcn = append(t_pcn,temp2014$PC.NO_2014[ii])
    t_pc09 = append(t_pc09,temp2009$PARLIAMENTARY.CONSTITUENCY_2009[ii])
    t_pc14 = append(t_pc14,temp2014$PARLIAMENTARY.CONSTITUENCY_2014[ii])
    
  }
}
mismatching_pc_elect_09_14 = data.frame(t_rn,t_stc, t_pcn, t_pc09, t_pc14)
print(mismatching_pc_elect_09_14)

data frame with 0 columns and 0 rows


##### There are no mismatches in PARLIAMENTARY.CONSTITUENCY name between 2009 and 2014

###### If it finds mismatches, here is the way to fix it

In [24]:
for(jj in 1:nrow(mismatching_pc_elect_09_14))
{
  LS2009Electors$PARLIAMENTARY.CONSTITUENCY[mismatching_pc_elect_09_14$t_rn[jj]] = LS2014Electors$PARLIAMENTARY.CONSTITUENCY[mismatching_pc_elect_09_14$t_rn[jj]]
}


### Electorals Merging

In [25]:
temp2009 = LS2009Electors
temp2014 = LS2014Electors
names(temp2009) = c("STATE.CODE_2009","STATE_2009","PC.NO_2009","PARLIAMENTARY.CONSTITUENCY_2009","Total.voters_2009","Total_Electors_2009","TOT_CONTESTANT_2009","POLL.PERCENTAGE_2009")
names(temp2014) = c("STATE.CODE_2014","STATE_2014","PC.NO_2014","PARLIAMENTARY.CONSTITUENCY_2014","Total.voters_2014","Total_Electors_2014","POLL.PERCENTAGE_2014")
temp = merge(y=temp2009,x=temp2014,by.y=c("STATE.CODE_2009","PC.NO_2009","PARLIAMENTARY.CONSTITUENCY_2009"), by.x = c("STATE.CODE_2014","PC.NO_2014","PARLIAMENTARY.CONSTITUENCY_2014"), all = T)

In [26]:
names(temp)
Electors_src = temp[,c(1,4,2,3,5,6,7,9,10,12,11)]

[1] "STATE.CODE_2014"                 "PC.NO_2014"                     
 [3] "PARLIAMENTARY.CONSTITUENCY_2014" "STATE_2014"                     
 [5] "Total.voters_2014"               "Total_Electors_2014"            
 [7] "POLL.PERCENTAGE_2014"            "STATE_2009"                     
 [9] "Total.voters_2009"               "Total_Electors_2009"            
[11] "TOT_CONTESTANT_2009"             "POLL.PERCENTAGE_2009"

### Candidate Data Cleaning

In [27]:
str(LS2009Candidates)

'data.frame':	2064 obs. of  14 variables:
 $ ST_CODE           : Factor w/ 35 levels "S01","S02","S03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ State.name        : Factor w/ 35 levels "Andaman & Nicobar Islands",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Month             : int  3 3 3 3 3 3 3 3 3 3 ...
 $ Year              : int  2009 2009 2009 2009 2009 2009 2009 2009 2009 2009 ...
 $ PC.Number         : int  1 1 1 1 1 1 1 1 1 2 ...
 $ PC.name           : Factor w/ 540 levels "Adilabad","Agra",..: 1 1 1 1 1 1 1 1 1 405 ...
 $ PC.Type           : Factor w/ 3 levels "GEN","SC","ST": 3 3 3 3 3 3 3 3 3 2 ...
 $ Candidate.Name    : Factor w/ 7786 levels "'AIDS MAN' PRAKASH TATERAO LANDGE",..: 5848 3376 4096 117 5851 930 2211 4585 688 1980 ...
 $ Candidate.Sex     : Factor w/ 3 levels "","F","M": 3 3 3 3 3 3 3 3 3 3 ...
 $ Candidate.Category: Factor w/ 4 levels "","GEN","SC",..: 4 4 4 4 4 4 4 4 4 3 ...
 $ Candidate.Age     : int  43 39 59 55 50 55 36 39 47 51 ...
 $ Party.Abbreviation: Factor w/ 364 levels "","A

In [28]:
LS2009Candidates$Candidate.Name = trimws(tolower(LS2009Candidates$Candidate.Name))
LS2009Candidates$Candidate.Name = sapply(LS2009Candidates$Candidate.Name,function(x) gsub('[ ]{2,}', ' ',x))
LS2009Candidates$Candidate.Name = sapply(LS2009Candidates$Candidate.Name,function(x) gsub('[[:punct:]]', '',x, perl = T))
head(LS2009Candidates$Candidate.Name,10)

[1] "rathod ramesh"        "kotnak ramesh"        "mesram nago rao"     
 [4] "ade tukaram"          "rathod sadashiv naik" "banka sahadevu"      
 [7] "ganta pentanna"       "nethavat ramdas"      "athram laxman rao"   
[10] "drgvivekanand"

In [29]:
LS2009Candidates$State.name = trimws(tolower(LS2009Candidates$State.name))
LS2009Candidates$PC.name = trimws(tolower(LS2009Candidates$PC.name))
LS2009Candidates$Candidate.Category = trimws(LS2009Candidates$Candidate.Sex)

##### Sort The data

In [30]:
LS2009Candidates = LS2009Candidates[with(LS2009Candidates,order(ST_CODE, PC.Number)),]


### Similarly do it fro 2014 also

In [31]:
str(LS2014Candidates)

'data.frame':	2306 obs. of  14 variables:
 $ ST_CODE           : Factor w/ 35 levels "S01","S02","S03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ State.name        : Factor w/ 35 levels "Andaman & Nicobar Islands",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Month             : int  5 5 5 5 5 5 5 5 5 5 ...
 $ Year              : int  2014 2014 2014 2014 2014 2014 2014 2014 2014 2014 ...
 $ PC.Number         : int  1 1 1 1 1 1 1 1 1 2 ...
 $ PC.name           : Factor w/ 540 levels "Adilabad ","Agra",..: 1 1 1 1 1 1 1 1 1 405 ...
 $ PC.Type           : Factor w/ 4 levels "GEN","SC","SC ",..: 4 4 4 4 4 4 4 4 4 2 ...
 $ Candidate.Name    : Factor w/ 7956 levels " ABDUL AMIR AMIRO",..: 2320 4549 5789 5934 4633 4708 4302 5004 957 927 ...
 $ Candidate.Sex     : Factor w/ 4 levels "F","M","NULL",..: 2 2 2 2 2 3 2 2 2 2 ...
 $ Candidate.Category: Factor w/ 5 levels "Gen","GEN","NULL",..: 5 5 5 5 5 3 5 5 5 4 ...
 $ Candidate.Age     : Factor w/ 63 levels "25","26","27",..: 25 13 24 31 20 63 16 9 29 7 ...
 $ Party.Abbrev

In [32]:
LS2014Candidates$Candidate.Name = trimws(tolower(LS2014Candidates$Candidate.Name))
LS2014Candidates$Candidate.Name = sapply(LS2014Candidates$Candidate.Name,function(x) gsub('[ ]{2,}', ' ',x))
LS2014Candidates$Candidate.Name = sapply(LS2014Candidates$Candidate.Name,function(x) gsub('[[:punct:]]', '',x, perl = T))


In [33]:
LS2014Candidates$State.name = trimws(tolower(LS2014Candidates$State.name))
LS2014Candidates$PC.name = trimws(tolower(LS2014Candidates$PC.name))
LS2014Candidates$Candidate.Category = trimws(LS2014Candidates$Candidate.Sex)

In [34]:
LS2014Candidates = LS2014Candidates[with(LS2014Candidates,order(LS2014Candidates$ST_CODE, LS2014Candidates$PC.Number)),]


#### Identify the people who contested in both the times

In [35]:
rn_2014 = c()
cname_2014 = c()


for(kk in 1:nrow(LS2014Candidates) )
{
  if(LS2014Candidates$Candidate.Name[kk] %in% LS2009Candidates$Candidate.Name)
  {
    rn_2014 = append(rn_2014,kk)
    cname_2014 = append(cname_2014,LS2014Candidates$Candidate.Name[kk])
      #s_dist = append(s_dist, d)
    
  }
}

contested_2009_2014 = as.data.frame(rn_2014, cname_2014)

### Getting the value from 2009 for merging it with 2014.

In [36]:
LS2009CandidatesFiltered = LS2009Candidates[contested_2009_2014$rn_2014,c('Candidate.Name','PC.name','Party.Abbreviation','Total.Votes.Polled','Position')]
dim(LS2009CandidatesFiltered)
head(LS2009CandidatesFiltered)

[1] 147   5

,Candidate.Name,PC.name,Party.Abbreviation,Total.Votes.Polled,Position
26,vinod kumar boinapally,karimnagar,TRS,267684,2
28,chandupatla janga reddy,karimnagar,BJP,122337,4
29,barige gattaiah yadav,karimnagar,IND,26026,5
37,gaddam raji reddy,karimnagar,IND,5205,13
47,dr vsathyanarayana murthy,nizamabad,PPOI,10296,8
54,malkapuram shiva kumar,zahirabad,PRAP,112792,3


In [37]:
temp = merge(x=LS2014Candidates,y=LS2009CandidatesFiltered,by='Candidate.Name',all.x = T)
Candidate_src = temp

In [38]:
names(Candidate_src)

[1] "Candidate.Name"       "ST_CODE"              "State.name"          
 [4] "Month"                "Year"                 "PC.Number"           
 [7] "PC.name.x"            "PC.Type"              "Candidate.Sex"       
[10] "Candidate.Category"   "Candidate.Age"        "Party.Abbreviation.x"
[13] "Total.Votes.Polled.x" "Position.x"           "PC.name.y"           
[16] "Party.Abbreviation.y" "Total.Votes.Polled.y" "Position.y"

In [39]:
names(Candidate_src) = c('Candidate.Name','ST_CODE','State.name','Month','Year','PC.Number','PC.Name','PC.Type','Candidate.Sex','Candidate.Category','Candidate.Age','Party.Abbreviation','Total.Votes.Polled','Position','PC.name.2009','Party.Abbreviation.2009','Total.Votes.Polled.2009','Position.2009')
Candidate_src = Candidate_src[,c(1,2,3,6,7,8,9,10,11,12,13,14,15,16,17,18)]

In [40]:
head(Candidate_src)

Candidate.Name,ST_CODE,State.name,PC.Number,PC.Name,PC.Type,Candidate.Sex,Candidate.Category,Candidate.Age,Party.Abbreviation,Total.Votes.Polled,Position,PC.name.2009,Party.Abbreviation.2009,Total.Votes.Polled.2009,Position.2009
a biju,S11,kerala,9,alathur,SC,M,M,36,IND,1692,12,NA,NA,NA,NA
a devdas,S10,karnataka,9,bellary,ST,M,M,45,SUCI,8486,5,NA,NA,NA,NA
a irudayadass,S22,tamil nadu,3,chennai south,GEN,M,M,53,IND,976,12,NA,NA,NA,NA
a jaiganesh,S22,tamil nadu,3,chennai south,GEN,M,M,29,LSP,518,16,NA,NA,NA,NA
a josekutty,S11,kerala,18,kollam,GEN,M,M,58,IND,1702,12,NA,NA,NA,NA
a magesh,S22,tamil nadu,3,chennai south,GEN,M,M,31,IND,121,41,NA,NA,NA,NA


## Merging Candidate data with Electoral Data

In [41]:
 Elector_Candidate = merge(x = Electors_src,y = Candidate_src, by.x = c('STATE.CODE_2014','PC.NO_2014'), by.y = c('ST_CODE','PC.Number'), all = T)

In [42]:
head(Elector_Candidate)

STATE.CODE_2014,PC.NO_2014,STATE_2014,PARLIAMENTARY.CONSTITUENCY_2014,Total.voters_2014,Total_Electors_2014,POLL.PERCENTAGE_2014,Total.voters_2009,Total_Electors_2009,POLL.PERCENTAGE_2009,...,Candidate.Sex,Candidate.Category,Candidate.Age,Party.Abbreviation,Total.Votes.Polled,Position,PC.name.2009,Party.Abbreviation.2009,Total.Votes.Polled.2009,Position.2009
S01,1,andhra pradesh,adilabad,1055593,1386282,76.15,864165,1131211,76.39,...,M,M,48,TDP,184198,3,NA,NA,NA,NA
S01,1,andhra pradesh,adilabad,1055593,1386282,76.15,864165,1131211,76.39,...,NULL,NULL,NULL,NOTA,17084,6,NA,NA,NA,NA
S01,1,andhra pradesh,adilabad,1055593,1386282,76.15,864165,1131211,76.39,...,M,M,49,TRS,430847,1,NA,NA,NA,NA
S01,1,andhra pradesh,adilabad,1055593,1386282,76.15,864165,1131211,76.39,...,M,M,55,BSP,94420,4,NA,NA,NA,NA
S01,1,andhra pradesh,adilabad,1055593,1386282,76.15,864165,1131211,76.39,...,M,M,33,IND,5055,8,NA,NA,NA,NA
S01,1,andhra pradesh,adilabad,1055593,1386282,76.15,864165,1131211,76.39,...,M,M,53,IND,4787,9,NA,NA,NA,NA


In [43]:
Elector_Candidate = Elector_Candidate[,c(1:12,15:25)]

In [44]:
Elector_Candidate = Elector_Candidate[,c(1,3,2,4,7,8,9,10,11,12,13,14,5,15,16,17,18,6,23,19,20,21,22)]

In [45]:
names(Elector_Candidate) = toupper(names(Elector_Candidate))
names(Elector_Candidate)

[1] "STATE.CODE_2014"                 "STATE_2014"                     
 [3] "PC.NO_2014"                      "PARLIAMENTARY.CONSTITUENCY_2014"
 [5] "POLL.PERCENTAGE_2014"            "TOTAL.VOTERS_2009"              
 [7] "TOTAL_ELECTORS_2009"             "POLL.PERCENTAGE_2009"           
 [9] "TOT_CONTESTANT_2009"             "CANDIDATE.NAME"                 
[11] "PC.TYPE"                         "CANDIDATE.SEX"                  
[13] "TOTAL.VOTERS_2014"               "CANDIDATE.CATEGORY"             
[15] "CANDIDATE.AGE"                   "PARTY.ABBREVIATION"             
[17] "TOTAL.VOTES.POLLED"              "TOTAL_ELECTORS_2014"            
[19] "POSITION.2009"                   "POSITION"                       
[21] "PC.NAME.2009"                    "PARTY.ABBREVIATION.2009"        
[23] "TOTAL.VOTES.POLLED.2009"

### Cleaning MyNeta Data

In [46]:
head(Candidate2014Declaration)

,Candidate,Constituency,Criminal.Cases,Education,ID,Party,Sno,Total.Assets,Total.Liabilities,Year,ST_CODE,PC_CODE
5,(Kuppal)G.Devadoss,CHENNAI SOUTH,0,8th Pass,6912,IND,5,3630000,850000,2014,S22,3
7,A Biju,ALATHUR,0,8th Pass,1906,IND,7,564425,175000,2014,S11,9
8,A N Radhakrishnan,ERNAKULAM,0,Graduate,8,BJP,8,7813475,0,2014,S11,12
9,A Nelson,SIVAGANGA,0,12th Pass,4337,IND,9,37322,0,2014,S22,31
10,A P Jithender Reddy,MAHBUBNAGAR,4,Graduate,4536,TRS,10,140831576,1386969,2014,S01,11
12,A Prasad,KOLAR,0,10th Pass,1105,IND,12,840000,0,2014,S10,28


In [47]:
Candidate2014Declaration$Candidate = trimws(tolower(Candidate2014Declaration$Candidate))
Candidate2014Declaration$Candidate = sapply(Candidate2014Declaration$Candidate,function(x) gsub('[ ]{2,}', ' ',x))
Candidate2014Declaration$Candidate = sapply(Candidate2014Declaration$Candidate,function(x) gsub('[[:punct:]]', '',x, perl = T))

In [48]:
head(Candidate2014Declaration)

,Candidate,Constituency,Criminal.Cases,Education,ID,Party,Sno,Total.Assets,Total.Liabilities,Year,ST_CODE,PC_CODE
5,kuppalgdevadoss,CHENNAI SOUTH,0,8th Pass,6912,IND,5,3630000,850000,2014,S22,3
7,a biju,ALATHUR,0,8th Pass,1906,IND,7,564425,175000,2014,S11,9
8,a n radhakrishnan,ERNAKULAM,0,Graduate,8,BJP,8,7813475,0,2014,S11,12
9,a nelson,SIVAGANGA,0,12th Pass,4337,IND,9,37322,0,2014,S22,31
10,a p jithender reddy,MAHBUBNAGAR,4,Graduate,4536,TRS,10,140831576,1386969,2014,S01,11
12,a prasad,KOLAR,0,10th Pass,1105,IND,12,840000,0,2014,S10,28


In [49]:
EC_Candidate = Elector_Candidate$CANDIDATE.NAME
MN_Candidate = Candidate2014Declaration$Candidate




In [50]:
unmatched = c()
t_sc_ec = c()
t_sc_mn = c()
t_pc_ec = c()
t_pc_mn = c()
t_mn = c()

for(l in 1:length(EC_Candidate))
    {
    if(!(EC_Candidate[l] %in% MN_Candidate))
        {
        t_sc_ec = append(t_sc_ec, Elector_Candidate$STATE.CODE_2014[l])
        t_sc_mn = append(t_sc_mn, Candidate2014Declaration$ST_CODE[l])
        t_pc_ec = append(t_pc_ec, Elector_Candidate$PC.NO_2014[l])
        t_pc_mn = append(t_pc_mn, Candidate2014Declaration$PC_CODE[l])
        t_mn = append(t_mn, Candidate2014Declaration$Candidate)
        unmatched = append(unmatched,EC_Candidate[l])
    }
}

unmatchedSet= data.frame(t_sc_ec, t_sc_mn, t_pc_ec, t_pc_mn, t_mn, unmatched)

In [51]:
unmatchedSet

t_sc_ec,t_sc_mn,t_pc_ec,t_pc_mn,t_mn,unmatched
1,11,1,9,kuppalgdevadoss,none of the above
1,10,10,9,a biju,patlolla kartik reddy
1,11,10,3,a n radhakrishnan,dr p sadhu satyanadhan
1,22,10,14,a nelson,none of the above
1,11,10,18,a p jithender reddy,samala ravindar
1,10,11,19,a prasad,ap jithender reddy
1,35,11,1,a sampath,none of the above
1,22,11,13,a thavamani,dr chandra sekhar a
1,22,12,14,a vijayaraghavan,yellaiah nandi
1,10,12,13,a devadas,buddula srinivas


In [ ]:
temp_unmatched = c()
temp_t_sc_ec = c()
temp_t_sc_mn = c()
temp_t_pc_ec = c()
temp_t_pc_mn = c()
temp_t_mn = c()

for(i1 in unmatchedSet$t_sc_ec)
    {
    if(i1 == unmatchedSet$t_sc_mn)
        {
        for(j1 in unmatchedSet$t_pc_ec)
                {
            if(j1 == unmatchedSet$t_pc_mn)
                {
                temp_t_sc_ec = append(temp_t_sc_ec, i1)
                temp_t_pc_ec = append(temp_t_pc_ec, j1)
                temp_t_mn = append(temp_t_mn,unmatchedSet$t_mn)
                temp_unmatched = append(temp_unmatched,unmatchedSet$unmatched)
            }
        }
    }
}

refnedUnmatched = data.frame(temp_t_sc_ec, temp_t_pc_ec, temp_t_mn, temp_unmatched)

Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning 

"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (i1 == unmatchedSet$t_sc_mn) {:
"the con

...

"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the con

"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the condition has length > 1 and only the first element will be used"Warning message in if (j1 == unmatchedSet$t_pc_mn) {:
"the con


# Tasks Pending
# Calculate Total Candidates by Aggrigating the data from LS2014Candidates and add it to Electors while merging.
